In [1]:
from init import *

In [2]:
def BuildModel_Simple():
    m = sdk.Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(sdk.IndependentGaussian())
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [3]:
m = BuildModel_Simple()
m.evaluations.append(sdk.EvaluationPoint(0,2))
m.json()

'{"TimeStart": 0, "TimeSteps": 10, "NumPaths": 10000, "updaters": [{"name": "IndependentGaussian", "refs": [], "args": []}, {"name": "BrownianMotion", "refs": [], "args": [0.2, 2], "start": 0.1}], "evaluations": [{"state": 0, "time": 2}]}'

In [4]:
r = requests.post(f'{server}/model',m.json())
r.json()

{'histograms': [],
 'mean': [0.5089091658592224],
 'names': ['BrownianMotion'],
 'npaths': [10000],
 'skewness': [0.008509947918355465],
 'stddev': [2.825547695159912],
 'time_points': [2.0],
 'time_steps': [10]}

In [5]:
import time
m = BuildModel_Simple()
m.NumPaths = 1000000
m.TimeSteps = 1000
m.RunTimeoutSeconds = 16.7
for t in [1,2,5,10]:
    m.evaluations.append(sdk.EvaluationPoint(0,t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'histograms': [],
 'mean': [0.2999381422996521,
  0.5017715692520142,
  1.1100157499313354,
  2.11881422996521],
 'names': ['BrownianMotion'],
 'npaths': [1000000, 1000000, 1000000, 1000000],
 'skewness': [0.0019918179605156183,
  -2.3200689611257985e-05,
  0.00023067706206347793,
  -0.00042090288479812443],
 'stddev': [2.0003764629364014,
  2.8320906162261963,
  4.473650932312012,
  6.3314337730407715],
 'time_points': [1.0, 2.0, 5.0, 10.0],
 'time_steps': [100, 200, 500, 1000]}

In [6]:
print(f'{m.NumPaths*m.TimeSteps/(time_end-time_start)*1e-9:.1f} nanoseconds to make a single time step')

0.5 nanoseconds to make a single time step


# Two processes, Three evaluations

In [7]:
def TwoProcessesModel ():
    m = sdk.Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(sdk.IndependentGaussian())
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [8]:
model = sdk.Model()
model.TimeStart = 0
model.TimeSteps = 10
model.NumPaths = 10000
model.Add(sdk.IndependentGaussian())
model.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.TimeSteps = 10000
for t in [1,2,5]:
    model.evaluations.append(sdk.EvaluationPoint(state=0,time=t))
r = requests.post(f'{server}/model',model.json())
r.json()

{'histograms': [],
 'mean': [0.3062770962715149,
  0.28895068168640137,
  0.4841310679912567,
  0.4786132872104645,
  1.1331366300582886,
  1.0854452848434448],
 'names': ['BrownianMotion', 'BrownianMotion'],
 'npaths': [10000, 10000, 10000, 10000, 10000, 10000],
 'skewness': [-0.016541460528969765,
  0.019555939361453056,
  -0.03647695854306221,
  -0.012951130047440529,
  -0.028081493452191353,
  0.041863538324832916],
 'stddev': [2.0030603408813477,
  2.014971971511841,
  2.8234856128692627,
  2.8345093727111816,
  4.496426582336426,
  4.518609046936035],
 'time_points': [1.0, 2.0, 5.0],
 'time_steps': [2000, 4000, 10000]}

In [9]:
results = run (model)
results.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,,0,1.0,2000,10000,0.307586,0.019951,1.995057,0.017630
1,BrownianMotion,,1,1.0,2000,10000,0.288734,0.019989,1.998921,-0.023256
2,BrownianMotion,,0,2.0,4000,10000,0.466456,0.028486,2.848572,-0.030302
3,BrownianMotion,,1,2.0,4000,10000,0.506796,0.028414,2.841415,-0.048010
4,BrownianMotion,,0,5.0,10000,10000,1.142684,0.044437,4.443688,-0.010366
5,BrownianMotion,,1,5.0,10000,10000,1.157495,0.045265,4.526543,-0.002890
